- Need to try to extract training data. 
- need to create a holdout to use to test the predictions with, can be a small holdout 
- there are possibly two different patterns - files from Christine 
- files that have been added since maybe the 4th of March

In [34]:
import glob
from xlrd import open_workbook
import os
import time
import random

In [35]:
ls b/

ls: b/: No such file or directory


In [36]:
clean_training_files = glob.glob("/Users/ianm/Dropbox/workbench/sage-maker-srm-test/clean-training-data/*")

In [37]:
excel_files = glob.glob("/Users/ianm/Dropbox/workbench/sage-maker-srm-test/excel-data/*")

# Excel helper functions

In [38]:
def files_from_team_member(initials, file_list):
    team_member_files = []
    for file in file_list:
        tail_name = file.split("/")[-1]
        if tail_name.split()[0] == initials:
            team_member_files.append(file)
    return team_member_files

In [39]:
def get_workbook(filename):
    wb = open_workbook(filename)
    return wb

In [40]:
def get_worksheet(filename, index):
    wb = open_workbook(filename)
    sheet = wb.sheets()[index]
    return sheet

In [41]:
def get_sheet_names(wb):
    return wb.sheet_names() 

In [42]:
def get_col_names(sheet):
    index_row = 0
    cols = sheet.row(index_row)
    col_names = [x.value for x in cols]
    return col_names

In [43]:
def list_sheets(excel_files):
    for excel in excel_files:
        wb = get_workbook(excel)
        sheets = get_sheet_names(wb)
        print(excel.split("/")[-1], sheets)
        print("")

In [111]:
def list_sheets_cols(excel_files):
    for excel in excel_files:
        wb = get_workbook(excel)
        sheets = get_sheet_names(wb)
        print(excel.split("/")[-1])
        for index, sheet_name in enumerate(sheets):
            sheet = wb.sheets()[index]
            col_names = get_col_names(sheet)
            non_empty_col_names = list(filter(lambda x: x != '', col_names))
            print(sheet_name, non_empty_col_names)
            print("-")
        print("")

### setup the sheet and col names for our clean training

all abstracts : Sheet1 / Ab 
accepted abstracts : Accepted / Abstract / Ab 
ignored : "do not chase", removed articles, Ignored

In [88]:
clean_training_accept = ["Accepted"]
clean_training_original = ["Sheet1", "Sheet 1"]
clean_training_ignored = ["do not chase", "removed articles", "Ignored"]
abstract_col_names = ["Ab", "Abstract", "AB"]

In [112]:
list_sheets_cols(clean_training_files)

CS American Journal of Industrial Medicine.xlsx
Accepted ['C1', 'First', 'Last', 'RP', 'Email', 'Title', 'Abstract', 'Remove?', 'Journal']
-
Sheet 1 ['C1', 'RP', 'EM', 'TI', 'AB', 'SO']
-
Ignored ['[Huntley, Samuel R.; Lee, David J.; LeBlanc, William G.; Arheart, Kristopher L.; McClure, Laura A.; Fleming, Lora E.; Caban-Martinez, Alberto J.] Univ Miami, Dept Publ Hlth Sci, Miller Sch Med, 1120 NW 14th St,R-669 Clin Res Bldg,Off 1025, Miami, FL 33136 USA; [Huntley, Samuel R.] Univ Miami, Miller Sch Med, Dept Orthopaed & Rehabil, Miami Ctr Orthopaed Res & Educ CORE, Miami, FL 33136 USA; [Fleming, Lora E.] Univ Exeter, European Ctr Environm & Human Hlth, Royal Cornwall Hosp, Knowledge Spa,Med Sch, Truro, Cornwall, England', 'Alberto', 'Caban-Martinez', 'Caban-Martinez, AJ (reprint author), Univ Miami, Dept Publ Hlth Sci, Miller Sch Med, 1120 NW 14th St,R-669 Clin Res Bldg,Off 1025, Miami, FL 33136 USA.', 'acaban@med.miami.edu', 'Acute joint pain in the emerging green collar workforce: Evi

## Explore structure of some of the excel fils from team members

In [89]:
cd_files = files_from_team_member("CS", excel_files)
kf_files = files_from_team_member("KF", excel_files)
mp_files = files_from_team_member("MP", excel_files)

## extract column data given sheet and column names

In [90]:
def check_sheet_names_against_validation(sheet_names, validation_list):
    for name in sheet_names:
        if name in validation_list: return True
    return False

In [101]:
def get_col_values_by_col_name(sheet, col_name_candidates):
    """
    the assumption is that we are looking for a colum from one sheet
    but the sheet and colum name might have one of a number of differnet values, but only one of them. 
    """
    col_names = get_col_names(sheet)
    for index, col_name in enumerate(col_names):
        if col_name in col_name_candidates:
            col_values = sheet.col_values(index)
            return col_values
    return None # if we don't find the column return None

In [118]:
def get_col_values_by_col_name_sheet_name(workbook, sheet_name_candidates, col_name_candidates):
    """
    the assumption is that we are looking for a colum from one sheet
    but the sheet and colum name might have one of a number of differnet values, but only one of them. 
    """
    sheet_names = get_sheet_names(workbook)
    print(sheet_names)
    for index, sheet_name in enumerate(sheet_names):
        if sheet_name in sheet_name_candidates:
            sheet = workbook.sheet_by_name(sheet_name)
            col_values = get_col_values_by_col_name(sheet, col_name_candidates)
            return col_values
    return None # if we don't find the sheet, return None 

## Get col data from clean traning data

In [103]:
def get_cols_from_files_by_colname(files, col_names):
    all_col_data = []
    for file in files:
        wb = get_workbook(file)
        col_data = get_col_values_by_col_name_sheet_name(wb, col_names, abstract_col_names)
        if col_data:
            all_col_data.extend(col_data)
    return all_col_data

In [119]:
ignored_clean_training = get_cols_from_files_by_colname(clean_training_files, clean_training_ignored)

['Accepted', 'Sheet 1', 'Ignored', 'cs edits - dups', 'removed articles']
['Accepted', 'Sheet1']
['Accepted', 'do not chase', 'Sheet1']
['Accepted', 'Sheet1']
['Accepted', 'Sheet1']
['Sheet1', 'Ignored', 'Accepted']


In [117]:
len(ignored_clean_training)

0

In [114]:
all_clean_training_abstracts = get_cols_from_files_by_colname(clean_training_files, clean_training_original)

accepted_clean_training = get_cols_from_files_by_colname(clean_training_files, clean_training_accept)

In [115]:
len(all_clean_training_abstracts), len(ignored_clean_training), len(accepted_clean_training) 

(1808, 0, 1192)

## clean up the "clean" training data

In [120]:
unique_all_clean_training_abstracts = list(set(all_clean_training_abstracts))
unique_accepted_clean_training_abstracts = list(set(accepted_clean_training))
len(unique_all_clean_training_abstracts), len(unique_accepted_clean_training_abstracts)

(1767, 1179)

In [122]:
# get the ones that have been rejected

In [123]:
unique_rejected_clean_training = []
for abstract in unique_all_clean_training_abstracts:
    if abstract not in unique_accepted_clean_training_abstracts:
        unique_rejected_clean_training.append(abstract)

In [127]:
len(unique_rejected_clean_training), len(unique_accepted_clean_training_abstracts)

(590, 1179)

In [125]:
%store unique_rejected_clean_training

Stored 'unique_rejected_clean_training' (list)


In [126]:
%store unique_accepted_clean_training_abstracts

Stored 'unique_accepted_clean_training_abstracts' (list)


## use abstract grabber to get abstracts from some of michaels files

In [59]:
abstract_col_names = ["AB", "Abstract"]
sheet_names_with_selections = ["Sheet1"]

In [61]:
selection_of_michaels_selected_abstracts = []
for file in mp_files[0:10]:
    wb = get_workbook(file)
    sheet_names_with_selections
    selected_abstracts = get_col_values_by_col_name_sheet_name(wb, sheet_names_with_selections, abstract_col_names)
    if selected_abstracts:
        selection_of_michaels_selected_abstracts.extend(selected_abstracts)
print(len(selection_of_michaels_selected_abstracts))

805


In [62]:
%store selection_of_michaels_selected_abstracts

Stored 'selection_of_michaels_selected_abstracts' (list)


# use abstract grabber to get abstracts from Christine's files

In [85]:
sheet_names_with_selections = ["contenders", "to email", "names to check for email", "double articles- manual email", "Contenders"]

In [86]:
abstract_col_names = ["AB", "Abstract"]
title_col_names = ["TI", "Title"]

In [25]:
# we do a double loop in this cell because we are looking for a selection from a sheet 
# that has accepted and rejceted abstracts 
all_test_abstracts = []
all_selected_abstracts = []
for file in christine_files:
    wb = get_workbook(file)
    original_sheet_candidates = ["original"]
    original_abstracts = get_col_values_by_col_name_sheet_name(wb, original_sheet_candidates, abstract_col_names)
    selected_abstracts = get_col_values_by_col_name_sheet_name(wb, sheet_names_with_selections, abstract_col_names)
    if original_abstracts and selected_abstracts:
        all_test_abstracts.extend(original_abstracts)
        all_selected_abstracts.extend(selected_abstracts)
        print(len(all_test_abstracts))
        print(len(all_selected_abstracts))
        print("")

NameError: name 'christine_files' is not defined

In [88]:
print(len(list(set(all_selected_abstracts))))

715


# let's remove duplicates.

In [89]:
unique_abstracts= list(set(all_test_abstracts))

In [90]:
unique_selected_abstracts = list(set(all_selected_abstracts))

In [91]:
rejected_abstracts = []
for abstract in unique_abstracts:
    if abstract not in unique_selected_abstracts:
        rejected_abstracts.append(abstract)

In [92]:
print(len(unique_abstracts))
print(len(unique_selected_abstracts))
print(len(rejected_abstracts))

1071
715
356


In [37]:
%store unique_selected_abstracts
%store rejected_abstracts

Stored 'unique_selected_abstracts' (list)
Stored 'rejected_abstracts' (list)


# scramble our lists and get our holdouts

In [154]:
random.shuffle(unique_selected_abstracts)
random.shuffle(rejected_abstracts)

In [157]:
HOLDOUT_NUMBER = 100
holdout_selected_abstracts = unique_selected_abstracts[0:HOLDOUT_NUMBER]
training_selected_abstracts = unique_selected_abstracts[HOLDOUT_NUMBER:]

In [158]:
print(len(unique_selected_abstracts))
print(len(holdout_selected_abstracts))
print(len(training_selected_abstracts))

715
100
615


In [5]:
%store unique_selected_abstracts

UsageError: Unknown variable 'unique_selected_abstracts'


# Next steps:  

- learn about what format SAGE maker needs for training 
- work though the two lists creating a tagged set of the format that SAGE maker needs  
- take out a 100 item set for both the selected and unselcted ones. 
- shuffle the training set  
- send the training set to SAGE maker   
- send the holdouts to SAGE maker to see how the model preforms  
- repeat the above for titles 